In [5]:
import pandas as pd
import re
from datetime import datetime

In [6]:
def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)

# veic_2019_02.csv_amostra

In [7]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

,index_df,x,y,data-hora,data,dia,mes,ano,tempo,hora,...,c,d,e,f,placa,h,i,j,k,l
0,index,n,x,tipo,data,dia,mes,ano,hora,y,...,vel2,k,s,cod,placa,r,t,y,u,i
1,1,501,1,2019-02-01 00:00:07,2019-02-01,01,02,2019,00:00:07,00,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
2,2,502,2,2019-02-01 00:00:09,2019-02-01,01,02,2019,00:00:09,00,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN
3,3,502,2,2019-02-01 00:00:17,2019-02-01,01,02,2019,00:00:17,00,...,61.0,2,3.6,bdac46ee2,LQM9654,0,0,NaN,NaN,NaN


# movimento_OCR_JAN_2019.txt_amostra.txt¶

In [8]:

# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

# Correção para linhas que começam com múltiplos espaços
def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)
            
# Separando as colunas manualmente
df_ocr['x'] = df_ocr.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_ocr['x'] = df_ocr.x.apply(lambda x: correctSplit(x))
df_ocr['placa'] = df_ocr.x.apply(lambda x: x.split('|')[0])
df_ocr['b'] = df_ocr.x.apply(lambda x: x.split('|')[1])
df_ocr['c'] = df_ocr.x.apply(lambda x: x.split('|')[2])
df_ocr['bairro'] = df_ocr.x.apply(lambda x: x.split('|')[3])
df_ocr['e'] = df_ocr.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_ocr.drop('x', axis = 1, inplace = True)

df_ocr.sample(3)

,placa,b,c,bairro,e
51,1MOTOCICLETA,20190101000020821024,0101201900002002400CET8-5433 0410281121025201...,St Est Cabucu,B-22.90235-43.53972
68,LPZ57203ÔNIBUS,20190101000027941014,0101201900002703600CET8-5408 0410101111025201...,St Recreio,A-22.94346-43.37263
10,LTD63411AUTOMÓVEL,20190101000003361015,0101201900000304100CET8-5991 0410501111025201...,St Campo Grande,A-22.98147-43.58786


# Tratando

In [9]:

def get_datahora(string):
    date = string[:14]
    return datetime.strptime(date, "%Y%m%d%H%M%S")

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_latitude(string):
    lat = string[1:10]
    return lat

def get_longitude(string):
    lon = string[10:19]
    return lon

df_ocr['DATAHORA'] = df_ocr['b'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr['placa'].apply(lambda x: get_placa(x))
df_ocr['LATITUDE'] = df_ocr['e'].apply(lambda x: get_latitude(x))
df_ocr['LONGITUDE'] = df_ocr['e'].apply(lambda x: get_longitude(x))
df_ocr['b'] = df_ocr['b'].apply(lambda x: x[14:])
df_ocr['c'] = df_ocr['c'].apply(lambda x: x[14:])
df_ocr.drop(['placa', 'bairro', 'e'], axis =1, inplace = True )
df_ocr['CGCT'] = df_ocr['c'].apply(lambda x: x.split()[1][:3])
df_ocr['CGPT'] = df_ocr['c'].apply(lambda x: x.split()[1][3:6])
df_ocr['CGLG'] = df_ocr['c'].apply(lambda x: x.split()[1][6:7])
df_ocr['CGPI'] = df_ocr['c'].apply(lambda x: x.split()[1][7:8])
df_ocr['CGST'] = df_ocr['c'].apply(lambda x: x.split()[1][8:9])
df_ocr['CGFX'] = df_ocr['c'].apply(lambda x: x.split()[1][9:10])
df_ocr['CODCET'] = df_ocr['c'].apply(lambda x: x.split()[1][:10])
df_ocr['c'] = df_ocr['c'].apply(lambda x: x.split()[0]+' '+x.split()[1][10:])
df_ocr['VELOCIDADE'] = df_ocr['c'].apply(lambda x: x[:3]+'.'+x[3:5])
df_ocr.drop(['b', 'c', 'CGPT', 'CGPI', 'CGST', 'CGFX', 'CGLG'], axis=1, inplace=True)
df_ocr.head(10)

,DATAHORA,PLACA,LATITUDE,LONGITUDE,CGCT,CODCET,VELOCIDADE
0,2019-01-01 00:00:00,KPV6877,-22.93380,-43.18294,041,0410191112,039.00
1,2019-01-01 00:00:01,LQR8247,-22.97019,-43.41377,041,0410121122,034.00
2,2019-01-01 00:00:01,ETP1890,-22.93149,-43.20888,041,0410241111,040.00
3,2019-01-01 00:00:01,KXL9916,-22.92479,-43.38907,041,0410161111,025.00
4,2019-01-01 00:00:01,LPU2287,-23.01267,-43.38900,041,0410381122,044.00
5,2019-01-01 00:00:02,LKU5609,-23.00850,-43.43940,041,0410551112,039.00
6,2019-01-01 00:00:02,KXL9916,-22.92479,-43.38907,041,0410161121,028.00
7,2019-01-01 00:00:02,,-22.79752,-43.19290,041,0410021111,023.00
8,2019-01-01 00:00:02,,-22.98010,-43.23462,041,0410451121,035.00
9,2019-01-01 00:00:03,KPH4077,-23.01267,-43.38900,041,0410381111,038.00


# SCP_INTVIA_0182018_201901_amostra.txt¶

In [10]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia = pd.read_csv('SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# Separando as colunas manualmente
df_intvia['x'] = df_intvia.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_intvia['x'] = df_intvia.x.apply(lambda x: correctSplit(x))
df_intvia['veiculo'] = df_intvia.x.apply(lambda x: x.split('|')[0])
df_intvia['a'] = df_intvia.x.apply(lambda x: x.split('|')[1])
df_intvia['b'] = df_intvia.x.apply(lambda x: x.split('|')[2])
df_intvia['c'] = df_intvia.x.apply(lambda x: x.split('|')[3])
df_intvia['d'] = df_intvia.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_intvia.drop('x', axis = 1, inplace = True)

df_intvia.head()

,veiculo,a,b,c,d
0,-------1moto,0101201900000215,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
1,LSY84841moto,0101201900000417,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
2,-------1moto,0101201900000714,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
3,-------1moto,0101201900002229,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
4,-------1moto,0101201900004443,BRI712,04800911100182018Rua Jardim Botanico,GAVEA


# SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt¶

In [11]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran = pd.read_csv('SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_sitran['x'] = df_sitran.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_sitran['x'] = df_sitran.x.apply(lambda x: correctSplit(x))
df_sitran['veiculo'] = df_sitran.x.apply(lambda x: x.split('|')[0])
df_sitran['a'] = df_sitran.x.apply(lambda x: x.split('|')[1])
df_sitran['b'] = df_sitran.x.apply(lambda x: x.split('|')[2])
df_sitran['c'] = df_sitran.x.apply(lambda x: x.split('|')[3])
df_sitran['d'] = df_sitran.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_sitran.drop('x', axis = 1, inplace = True)

df_sitran.head()

,veiculo,a,b,c,d
0,-------1moto,0103201910263053,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
1,-------1moto,0103201910263141,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
2,-------1moto,0103201910263449,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
3,LQS56581moto,0103201910490948,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
4,FLT20441moto,0103201910491046,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO


# SCP_TRANSCARIOCA_0222016_201903_amostra.txt¶

Previamente a minha coluna que continham os dados de latitude e longitude não constava no dataframe, então talvez haja a necessidade de criar uma nova coluna (no meu caso a coluna 'e') para depois conseguirmos tratar esses dados. Achei válido reiterar porque estava me gerando confusão. Vale fazer um check com o arquivo txt para verificar se existe algum dado que talvez se pareça com coordenadas de lat e long.

In [17]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca = pd.read_csv('SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_transcarioca['x'] = df_transcarioca.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_transcarioca['x'] = df_transcarioca.x.apply(lambda x: correctSplit(x))
df_transcarioca['veiculo'] = df_transcarioca.x.apply(lambda x: x.split('|')[0])
df_transcarioca['a'] = df_transcarioca.x.apply(lambda x: x.split('|')[1])
df_transcarioca['b'] = df_transcarioca.x.apply(lambda x: x.split('|')[2])
df_transcarioca['c'] = df_transcarioca.x.apply(lambda x: x.split('|')[3])
df_transcarioca['d'] = df_transcarioca.x.apply(lambda x: x.split('|')[4])
df_transcarioca['e'] = df_transcarioca.x.apply(lambda x: x.split('|')[5])

# Dropando a coluna x utilizada para separar as outras colunas
df_transcarioca.drop('x', axis = 1, inplace = True)

df_transcarioca.head(10)

,veiculo,a,b,c,d,e
0,-------2carro de passeio,010320190010319,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
1,-------1moto,0103201900104424,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
2,-------2carro de passeio,0103201900104419,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
3,-------1moto,0103201900104628,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
4,-------1moto,0103201900104830,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
5,-------1moto,0103201900105030,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
6,-------1moto,0103201900105834,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
7,-------1moto,0103201900110732,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
8,-------1moto,0103201900110833,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
9,-------1moto,0103201900111134,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"


In [18]:
def get_datahora(string):
    date = string[:14]
    return datetime.strptime(date, "%d%m%Y%H%M%S")

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_latitude(string):
    lat = string[1:4]+"."+string[5:10]
    return lat

def get_longitude(string):
    lon = string[10:13]+"."+string[14:19]
    return lon

df_transcarioca['DATAHORA'] = df_transcarioca['a'].apply(lambda x: get_datahora(x))
df_transcarioca['PLACA'] = df_transcarioca['veiculo'].apply(lambda x: get_placa(x))
df_transcarioca['VELOCIDADE'] = df_transcarioca['a'].str[14:]+".00"
df_transcarioca['LATITUDE'] = df_transcarioca['e'].apply(lambda x: get_latitude(x))
df_transcarioca['LONGITUDE'] = df_transcarioca['e'].apply(lambda x: get_longitude(x))
df_transcarioca['CODCET'] = df_transcarioca['c'].str[:10]
df_transcarioca['CGCT'] = df_transcarioca['c'].str[:3]
df_transcarioca['CGPT'] = df_transcarioca['c'].str[3:6]
df_transcarioca['CGLG'] = df_transcarioca['c'].str[6:7]
df_transcarioca['CGPI'] = df_transcarioca['c'].str[7:8]
df_transcarioca['CGST'] = df_transcarioca['c'].str[8:9]
df_transcarioca['CGFX'] = df_transcarioca['c'].str[9:10]
df_transcarioca.drop(['veiculo', 'a', 'b', 'c', 'd', 'e', 'CGPT', 'CGPI', 'CGST', 'CGFX', 'CGLG'], axis=1, inplace=True)
df_transcarioca.head(20)

,DATAHORA,PLACA,VELOCIDADE,LATITUDE,LONGITUDE,CODCET,CGCT
0,2019-03-01 00:10:31,,9.00,-22.87280,-43.33629,0440021120,044
1,2019-03-01 00:10:44,,24.00,-22.87280,-43.33629,0440021120,044
2,2019-03-01 00:10:44,,19.00,-22.87280,-43.33629,0440021120,044
3,2019-03-01 00:10:46,,28.00,-22.87280,-43.33629,0440021120,044
4,2019-03-01 00:10:48,,30.00,-22.87280,-43.33629,0440021120,044
5,2019-03-01 00:10:50,,30.00,-22.87280,-43.33629,0440021120,044
6,2019-03-01 00:10:58,,34.00,-22.87280,-43.33629,0440021120,044
7,2019-03-01 00:11:07,,32.00,-22.87280,-43.33629,0440021120,044
8,2019-03-01 00:11:08,,33.00,-22.87280,-43.33629,0440021120,044
9,2019-03-01 00:11:11,,34.00,-22.87280,-43.33629,0440021120,044


# SPLICE_007201820190205.txt_amostra.txt¶

In [19]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_splice = pd.read_csv('SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_splice['x'] = df_splice.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_splice['x'] = df_splice.x.apply(lambda x: correctSplit(x))
df_splice['veiculo'] = df_splice.x.apply(lambda x: x.split('|')[0])
df_splice['a'] = df_splice.x.apply(lambda x: x.split('|')[1])
df_splice['b'] = df_splice.x.apply(lambda x: x.split('|')[2])
df_splice['c'] = df_splice.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
df_splice.drop('x', axis = 1, inplace = True)

df_splice.sample(5)

,veiculo,a,b,c
42,1MOTO,2019010100024905800RD3400,04505711110072018R PRIMEIRO MARCO PX36,Candelaria
96,1MOTO,2019010100082005900RD3377,04502911120072018R SAO CLEMENTE PX 206,Humaita
62,1MOTO,2019010100043404800RD3384,04504411130072018R PRUDENTE MORAIS PX 690 FXBRS,Sao Conrado
6,1MOTO,2019010100002704200RD3390,04503911120072018AV GAL SAN MARTIN PX 470,Sao Conrado
4,1MOTO,2019010100002205800RD3385,04504511120072018R PRUDENTE MORAIS PX 938,Sao Conrado


# FISCALTECH_0042016_06_2019.txt_amostra.txt

In [20]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech = pd.read_csv('FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_fiscaltech['x'] = df_fiscaltech.x.replace({'( ){3,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_fiscaltech['x'] = df_fiscaltech.x.apply(lambda x: correctSplit(x))
df_fiscaltech['a'] = df_fiscaltech.x.apply(lambda x: x.split('|')[0])
df_fiscaltech['b0'] = df_fiscaltech.x.apply(lambda x: x.split('|')[1])
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: ' '.join(list(x.split(' ')[1:])))
df_fiscaltech['d'] = df_fiscaltech.x.apply(lambda x: x.split('|')[2])
# df_fiscaltech['c'] = df_fiscaltech.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
df_fiscaltech.drop(['x','b0'], axis = 1, inplace = True)

df_fiscaltech.sample(5)

,a,b,c,d
38,3,0106201903295104900FSCII2209,04201612220042016AvSalvAllende Px6300-PLat-St...,2
98,1,0106201901140505800FSCII2187,04201111220042016AvSalvAllende Px4755-PCent-S...,2-22963742-43394364
1,1,0106201901164305100FSCII2214,04201712110042016AvSalvAllende Px5400-PLat-St...,1
26,3,0106201905140403600FSCII2195,04201811220042016AvAbelardoBueno Px3500 PCent...,2-22973215-43397949
10,AUN60091,0106201903315208200FSCII2199,04201311220042016AvAbelardoBueno Px980 PCent-...,2-22973217-43387464
